In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import config as cfg
import os
import pascal_voc as pascl
import numpy as np
import tensorflow as tf
import network
import datetime
import cv2
from nms import py_cpu_nms
import matplotlib.pyplot as plt
from PIL import Image

from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers.tensorflow import deploy_tensorflow_model
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()
clipper_conn.connect()



clipper_conn.register_application(
    name="faster-app", input_type="strings", default_output="-1.0", slo_micros=100000)



/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



20-06-28:05:07:04 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-06-28:05:07:08 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmp7d9mf28a.yml
20-06-28:05:07:10 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running
20-06-28:05:07:10 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337
20-06-28:05:07:10 INFO     [clipper_admin.py:236] [default-cluster] Application faster-app was successfully registered


In [ ]:
def predict(sess,train_data):
    import numpy as np
    num_imgs = len(train_data)
#     print('这是啥',train_data,num_imgs)
    result = []
    for i in range(num_imgs):
        data = json.loads(train_data[i][0])
        print(data,type(data),type(i),type(train_data))
#         test_data[i] = data
        image_height = np.array(np.array(data['image']).shape[1])
        image_width = np.array(np.array(data['image']).shape[2])
        feed_dict = {"Placeholder:0": data['image'], "Placeholder_1:0": image_width,\
                     "Placeholder_2:0": image_height}
        rois_coord, pred_box, pred_box_score_arg, pred_score= sess.run(["strided_slice_13:0", "vgg_16_2/region_deciton/bbox_pred/BiasAdd:0",
                                                                        "ArgMax:0", "vgg_16_2/region_deciton/cls_prob:0"],\
                                                                feed_dict=feed_dict)
        result.extend([str(rois_coord), str(pred_box), str(pred_box_score_arg), str(pred_score)])
    return result

#"log":"TypeError: list indices must be integers or slices, not str\n","stream":"stderr","time":"2020-06-28T02:46:45.704169109Z"}


In [2]:
#此处可看出 输入的最终数据为  [{"image": [[1, 2, 3]]}] 
#clipper 接收到的数据为 [b'[{"image": [[1, 2, 3]]}]' b'[{"image": [[1, 2, 3]]}]']

def predict2(sess,train_data):
    import numpy as np
#     num_imgs = len(train_data)
#     print('这是啥',train_data,num_imgs)
    result = []
#     for i in range(num_imgs):
    data = json.loads(train_data[0])[0]
    print(data,type(data))
#         test_data[i] = data
    image_height = np.array(np.array(data['image']).shape[1])
    image_width = np.array(np.array(data['image']).shape[2])
    feed_dict = {"Placeholder:0": data['image'], "Placeholder_1:0": image_width,\
                 "Placeholder_2:0": image_height}
    rois_coord, pred_box, pred_box_score_arg, pred_score= sess.run(["strided_slice_13:0", "vgg_16_2/region_deciton/bbox_pred/BiasAdd:0",
                                                                    "ArgMax:0", "vgg_16_2/region_deciton/cls_prob:0"],\
                                                            feed_dict=feed_dict)
    result.extend([str(rois_coord), str(pred_box), str(pred_box_score_arg), str(pred_score)])
    return result

In [ ]:
list1 = []
list2 = [str([1]),str([2])]
list1.extend(list2)
print(type(list1))

In [3]:
import matplotlib.pyplot as plt
import base64
import json
import requests
from PIL import Image
from datetime import datetime

import time

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, time):
            return obj.__str__()
        else:
            return super(NpEncoder, self).default(obj)


class Val_test(object):   
    def __init__(self, net ,val_data):
        self.net = net
        self.val_data = val_data
        self.overlaps_max = cfg.overlaps_max
#         self.graph = tf.get_default_graph()
        self.overlaps_min = cfg.overlaps_min
        self.ckpt_filename = tf.train.latest_checkpoint(cfg.OUTPUT_DIR)
        self.test_output_dir = cfg.test_output_path
        self.image_output_dir = cfg.image_output_dir
        
        
    def test_model(self):
        saver = tf.train.Saver()
#         saver = tf.train.import_meta_graph('/root/code/Faster-rcnn-tensorflow-master_1/output/output.model-70000.meta')

        _rois_coord = self.net.rois_coord[:,1:5]
        #rois_coord = self.net.rois_coord
        _pred_box = self.net.bbox_pred
        _pred_score = self.net.cls_prob
        _pred_box_score_arg = tf.argmax(_pred_score, axis=1)
        dect_total_result = [[[] for i in range(cfg.img_save_num)] for j in range(self.net.num_classes)]
        test_data = [[] for i in range (cfg.img_save_num)]
        with tf.Session() as sess:
            saver.restore(sess, self.ckpt_filename)
            
            print('所有的变量名字 在这里 ')
#             print([n.name for n in graph.as_graph_def().node])

#             for node in sess.graph_def.node:
#                 print(node)
            print('结束了')
            #部署clipper预测容器 
            
            deploy_tensorflow_model(clipper_conn,name="faster-mod",version=1,input_type="strings",func=predict2,
            tf_sess_or_saved_model_path=sess,pkgs_to_install=['numpy'])
            
            clipper_conn.link_model_to_app(app_name="faster-app",model_name="faster-mod")
            
            query_address = clipper_conn.get_query_addr()
            
            url = "http://%s/faster-app/predict" %  query_address
            
#             for i in range (cfg.img_save_num):
#                 print (i, ' image test compeleted')            
            train_data = self.val_data.get()  #if you want to change the test imgae, you can using cv2.imread() here to read your own image data


            print('这是测试数据哦',train_data)
            tva = json.dumps([train_data],cls=MyEncoder)

            tva_j = json.dumps({"input": tva})
            headers = {'Content-type': 'application/json'}
            start = datetime.now()
            print('睡10秒再请求')
            

        
        
        
#             return None

        
            time.sleep(10)
            r = requests.post(url, headers=headers, data=tva_j)
            end = datetime.now()
            latency = (end - start).total_seconds() * 1000.0
            re = r.text

            print(re)
            return re


# #             os.exit()

#             pred_box_score_arg = pred_box_score_arg.astype(np.int32)
#             num_pred = pred_box_score_arg.shape[0]
#             pred_box_gather = np.empty([num_pred, 4], dtype = np.float32)
#             pred_score_gather = np.empty(num_pred)

#             for j in range(num_pred):
#                 pred_box_gather[j, :] = pred_box[j, 4*pred_box_score_arg[j]:4*(pred_box_score_arg[j]+1)]
#                 pred_score_gather[j] = pred_score[j, pred_box_score_arg[j]]

#             pred_box_gather = pred_box_gather * np.array(cfg.bbox_nor_stdv) + np.array(cfg.bbox_nor_mean)
#             pre_box_coord = self.coord_transform_inv(rois_coord, pred_box_gather.astype(np.float32))
#             pre_box_coord = pre_box_coord
#             for k in range(1, self.net.num_classes):
#                 pre_class_arg = np.where(pred_box_score_arg==k)[0]
#                 cls_pred_box_coord = pre_box_coord[pre_class_arg, :]
#                 cls_pred_score = pred_score_gather[pre_class_arg]
#                 #print(cls_pred_box_coord.shape, cls_pred_score.shape)
#                 cls_pred_score = cls_pred_score[:, np.newaxis]  
#                 cls_pred_target = np.concatenate((cls_pred_box_coord, cls_pred_score), axis=1)
#                 keep = py_cpu_nms(cls_pred_target, cfg.test_nms_thresh)
#                 cls_pred_target = cls_pred_target[keep, :]
#                 dect_total_result[k][i] = cls_pred_target

#             for k in range(cfg.img_save_num):
#                 imname = test_data[k]['imname']
#                 im_scale = test_data[k]['scale']
#                 image = cv2.imread(imname)
#                 im = cv2.resize(image, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
#                 im = self.draw_result(im, dect_total_result, k)
#                 if not os.path.exists(self.image_output_dir):
#                     os.mkdir(self.image_output_dir)
#                 im_save_path = os.path.join(self.image_output_dir, '{:d}'.format(k)+'.jpg')
#                 cv2.imwrite(im_save_path, im)
                
                
# #                 cv2.imshow('Image',im)#这里有环境问题
# #                 cv2.destroyAllWindows()
#                 plt.figure(figsize=(10,10))
        
#                 im = Image.open(im_save_path)
#                 plt.imshow(im)  #显示需要识别的图片
#                 plt.show()
                
                
                
                
#                 cv2.waitKey(0)


    def draw_result(self, img, result, ind):
        for i in range(1, self.net.num_classes):
            for j in range(result[i][ind].shape[0]):
                if result[i][ind][j][4]>0.5:
                    x1 = int(result[i][ind][j][0])
                    y1 = int(result[i][ind][j][1])
                    x2 = int(result[i][ind][j][2])
                    y2 = int(result[i][ind][j][3])                    
                    cv2.rectangle(img, (x1, y1), (x2, y2), ((i%19)*15, (i%3)*100+40, (i%6)*50+35), 2) 
                    cv2.rectangle(img, (x1, y1-20),(x2, y1), (125, 125, 125), -1)
                    lineType = cv2.LINE_AA if cv2.__version__ > '3' else cv2.CV_AA
                    cv2.putText(img, cfg.CLASSES[i] + ' : %.2f' % result[i][ind][j][4],\
                    (x1 + 5, y1 - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.5,\
                    (0, 0, 0), 1, lineType)                
        return img
        
        
    def coord_transform_inv (self, anchors, boxes):
        anchors = anchors.astype(np.float32)
        anchors = np.reshape(anchors, [-1,4])
        anchor_x = (anchors[:,2] + anchors[:,0]) * 0.5
        anchor_y = (anchors[:,3] + anchors[:,1]) * 0.5
        acnhor_w = (anchors[:,2] - anchors[:,0]) + 1.0
        acnhor_h = (anchors[:,3] - anchors[:,1]) + 1.0
        boxes = np.reshape(boxes, [-1,4])
        boxes_x = boxes[:,0]*acnhor_w + anchor_x
        boxes_y = boxes[:,1]*acnhor_h + anchor_y
        boxes_w = np.exp(boxes[:,2])*acnhor_w
        boxes_h = np.exp(boxes[:,3])*acnhor_h
        coord_x1 = boxes_x - boxes_w*0.5
        coord_y1 = boxes_y - boxes_h*0.5
        coord_x2 = boxes_x + boxes_w*0.5
        coord_y2 = boxes_y + boxes_h*0.5
        coord_result = np.stack([coord_x1, coord_y1, coord_x2, coord_y2], axis=1)
        return coord_result                  
    
    
#get variables_to_restore               
    def get_var_list(self, global_variables, ckpt_variables):
        variables_to_restore = []
        for key in global_variables:
            if key.name.split(':')[0] in ckpt_variables:
                variables_to_restore.append(key) 
        return variables_to_restore
    
    

if __name__ == '__main__':
    os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID
#     import os
#     os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
    net = network.Net(is_training=False)
    val_data = pascl.pascal_voc(cfg.test_imdb_name, 'test', fliped=False)
    test = Val_test(net, val_data)
    print ('start training')
    test.test_model()
#     sess = test.test_model()#把sess传出来

20-06-28:05:07:34 WARNING  [module_wrapper.py:139] From /root/code/Faster-rcnn-tensorflow-master_1/network.py:28: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



20-06-28:05:07:34 WARNING  [module_wrapper.py:139] From /root/code/Faster-rcnn-tensorflow-master_1/network.py:104: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



Instructions for updating:
Please use `layer.__call__` method instead.


20-06-28:05:07:34 WARNING  [deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


...

20-06-28:05:07:35 WARNING  [deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.


20-06-28:05:07:36 WARNING  [module_wrapper.py:139] From /root/code/Faster-rcnn-tensorflow-master_1/network.py:267: The name tf.summary.image is deprecated. Please use tf.compat.v1.summary.image instead.



Loading gt_labels from: annotation_cache/VOC2007_test/pascal_test_gt_labels.pkl
start training
INFO:tensorflow:Restoring parameters from output/output.model-70000


20-06-28:05:07:36 INFO     [saver.py:1284] Restoring parameters from output/output.model-70000
20-06-28:05:07:37 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpx2baqe9xclipper
20-06-28:05:07:37 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-06-28:05:07:38 INFO     [tensorflow.py:196] TensorFlow model saved at: /tmp/tmpx2baqe9xclipper/tfmodel/model.ckpt 
20-06-28:05:07:38 INFO     [tensorflow.py:277] Using Python 3.6 base image
20-06-28:05:07:39 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpx2baqe9xclipper
20-06-28:05:07:52 INFO     [clipper_admin.py:539] [default-cluster] Step 1/3 : FROM clipper/tf36-container:0.4.1
20-06-28:05:07:52 INFO     [clipper_admin.py:539] [default-cluster]  ---> 3db42af800ff
20-06-28:05:07:52 INFO     [clipper_admin.py:539] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential && pip install numpy
20-06-28:05:07:52 INFO     [clipper_admin.py:539

这是测试数据哦 {'image': array([[[[-121.7717  , -114.9465  , -102.9801  ],
         [-121.7717  , -114.9465  , -102.9801  ],
         [-121.7717  , -114.9465  , -102.9801  ],
         ...,
         [-120.80087 , -111.97568 , -100.00928 ],
         [-121.38919 , -112.563995, -100.597595],
         [-121.7717  , -112.9465  , -100.9801  ]],

        [[-121.7717  , -114.9465  , -102.9801  ],
         [-121.7717  , -114.9465  , -102.9801  ],
         [-121.7717  , -114.9465  , -102.9801  ],
         ...,
         [-120.80087 , -111.97568 , -100.00928 ],
         [-121.38919 , -112.563995, -100.597595],
         [-121.7717  , -112.9465  , -100.9801  ]],

        [[-121.7717  , -114.9465  , -102.9801  ],
         [-121.7717  , -114.9465  , -102.9801  ],
         [-121.7717  , -114.9465  , -102.9801  ],
         ...,
         [-120.80087 , -111.97568 , -100.00928 ],
         [-121.38919 , -112.563995, -100.597595],
         [-121.7717  , -112.9465  , -100.9801  ]],

        ...,

        [[-108.23404

In [ ]:
val_data = pascl.pascal_voc(cfg.test_imdb_name, 'test', fliped=False)
query_address = clipper_conn.get_query_addr()

url = "http://%s/faster-app/predict" %  query_address

#             for i in range (cfg.img_save_num):
#                 print (i, ' image test compeleted')            
train_data = val_data.get()  #if you want to change the test imgae, you can using cv2.imread() here to read your own image data

print(len(train_data))

#clipper要求输入必须为 字符串 故必须要序列化下
# list1 = {'image':[[1,2,3]]}
tva = json.dumps([train_data],cls=MyEncoder)
# tva = json.dumps([list1],cls=MyEncoder)
# tva = [train_data]

print(tva)
tva_j = json.dumps({"input": tva})
print(tva_j)
#此处可看出 输入的最终数据为  [{"image": [[1, 2, 3]]}] 
#clipper 接收到的数据为 [b'[{"image": [[1, 2, 3]]}]' b'[{"image": [[1, 2, 3]]}]']
headers = {'Content-type': 'application/json'}
start = datetime.now()
r = requests.post(url, headers=headers, data=tva_j)
end = datetime.now()
latency = (end - start).total_seconds() * 1000.0
re = r.text

print(re)

#docker 日志显示 ValueError: callback pyfunc_0 is not found 报错 疑似 线 程 问 题  
#原文件在预测时 是在同一个线程 但是 我们把sess部署到clipper上 时线程改变了 故找不到相关参数 
# 正在找解决办法

In [ ]:
print(len([train_data]))
print([train_data][0])

In [ ]:
def predict1(sess,train_data):
    import numpy as np
    num_imgs = len(train_data)
#     print('这是啥',train_data,num_imgs)
    result = []
    for i in range(num_imgs):
        data =train_data
#         data = json.loads(train_data[i])
#         print(data,type(data),type(i),type(train_data))
#         test_data[i] = data
        image_height = np.array(data['image'].shape[1])
        image_width = np.array(data['image'].shape[2])
        feed_dict = {"Placeholder:0": data['image'], "Placeholder_1:0": image_width,\
                     "Placeholder_2:0": image_height}
        rois_coord, pred_box, pred_box_score_arg, pred_score= sess.run(["strided_slice_13:0", "vgg_16_2/region_deciton/bbox_pred/BiasAdd:0",
                                                                        "ArgMax:0", "vgg_16_2/region_deciton/cls_prob:0"],\
                                                                feed_dict=feed_dict)
        result.extend([str(rois_coord), str(pred_box), str(pred_box_score_arg), str(pred_score)])
    return result

In [ ]:
val_data = pascl.pascal_voc(cfg.test_imdb_name, 'test', fliped=False)
train_data = val_data.get() 
predict1(sess,train_data)

In [ ]:
sess

In [4]:
clipper_conn.stop_all()

20-06-28:23:13:21 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers
